In [1]:
import spacy as spacy
#check https://towardsdatascience.com/auto-generated-knowledge-graphs-92ca99a81121
from spacy.matcher import Matcher 
from spacy.tokens import Span 
import pandas as pd
import re

In [2]:
#approach
#first split the article/paragraph into sentences 
#second extract entity from text using part-of-speech & dependency parsing(used for entities that areCompound words)
#extract relation using the ROOT of the sentence

In [3]:
nlp = spacy.load('en_core_web_md')

In [4]:
#read text file and split article into sentences

text = ("Competitors to Office include software and global application vendors, such as Apple, Cisco Systems, Facebook, Google, IBM, Okta, Proofpoint, Slack, Symantec, Zoom, and numerous web-based and mobile application competitors as well as local application developers. Apple distributes versions of its pre-installed application software, such as email and calendar products, through its PCs, tablets, and phones. Cisco Systems is using its position in enterprise communications equipment to grow its unified communications business. Google provides a hosted messaging and productivity suite. Slack provides teamwork and collaboration software. Zoom offers videoconferencing and cloud phone solutions. Skype for Business and Skype also compete with a variety of instant messaging, voice, and video communication providers, ranging from start-ups to established enterprises. Okta, Proofpoint, and Symantec provide security solutions across email security, information protection, identity, and governance. Web-based offerings competing with individual applications have also positioned themselves as alternatives to our products and services. We compete by providing powerful, flexible, secure, integrated industry-specific, and easy-to-use productivity and collaboration tools and services that create comprehensive solutions and work well with technologies our customers already have both on-premises or in the cloud. LinkedIn faces competition from online recruiting companies, talent management companies, and larger companies that are focusing on talent management and human resource services; job boards; traditional recruiting firms; and companies that provide learning and development products and services. Marketing Solutions competes with online and offline outlets that generate revenue from advertisers and marketers. Dynamics competes with vendors such as Infor, NetSuite, Oracle, Salesforce.com, SAP, and The Sage Group to provide cloud-based and on-premise business solutions for small, medium, and large organizations.")
text2 = open('sample_KG.txt')
doc = nlp(text2.read())

#get sentences
sentences = list(doc.sents)

#print(sentences[:20])


#print dependency tag
#for tok in sentences[5]:
    #print(tok.text, "...", tok.dep_)
#print entities NER
for ent in doc.ents:
    if ent.label_=='ORG' or ent.label_=='PRODUCT' or ent.label_=='PERSON':
        print(ent.text, ent.start_char, ent.end_char, ent.label_)

Microsoft 48 57 ORG
Microsoft Office 2846 2862 ORG
Microsoft Surface 2905 2922 PRODUCT
Microsoft 2974 2983 ORG
365 2984 2987 PRODUCT
Office 3151 3157 ORG
Microsoft Dynamics 3159 3177 ORG
LinkedIn 3183 3191 ORG
Microsoft 3193 3202 ORG
Enterprise Mobility + Security 3251 3281 ORG
AI 3333 3335 ORG
Microsoft Teams 3477 3492 ORG
Microsoft Relationship Sales 3640 3668 ORG
LinkedIn 3694 3702 ORG
Sales 3703 3708 PRODUCT
Dynamics 3723 3731 PRODUCT
Dynamics 365 3796 3808 PRODUCT
Talent 3813 3819 PRODUCT
the Intelligent Cloud Platform 4322 4352 PRODUCT
Microsoft 4881 4890 ORG
Azure Stack 5686 5697 PRODUCT
SQL Database 6126 6138 ORG
Microsoft SQL Server 6179 6199 ORG
AI 6416 6418 ORG
Azure Machine Learning Studio 6463 6492 PRODUCT
Azure Machine Learning Workbench 6554 6586 ORG
Microsoft 6676 6685 ORG
GitHub, Inc. 6713 6725 ORG
Cortana 7443 7450 ORG
OneDrive 7599 7607 ORG
Eye Control 7691 7702 ORG
OEM 7904 7907 ORG
Windows 8206 8213 PRODUCT
Windows 10 8232 8242 PRODUCT
Xbox Live 8960 8969 ORG
Windo

In [ ]:
#second extract entity from text using part-of-speech & dependency parsing(used for entities that areCompound words)
def get_entities(sent):
    ## chunk 1
    ent1 = ""
    ent2 = ""
    
    prv_tok_dep = ""    # dependency tag of previous token in the sentence
    prv_tok_text = ""   # previous token in the sentence
    
    prefix = ""
    modifier = ""
    
    #############################################################
    
    for tok in sent:
        ## chunk 2
        # if token is a punctuation mark then move on to the next token
        if tok.dep_ != "punct":
            # check: token is a compound word or not
            if tok.dep_ == "compound":
                prefix = tok.text
                # if the previous word was also a 'compound' then add the current word to it
                if prv_tok_dep == "compound":
                    prefix = prv_tok_text + " "+ tok.text
            # check: token is a modifier or not
            if tok.dep_.endswith("mod") == True:
                modifier = tok.text
                # if the previous word was also a 'compound' then add the current word to it
                if prv_tok_dep == "compound":
                    modifier = prv_tok_text + " "+ tok.text
                    
        ## chunk 3
        if tok.dep_.find("subj") == True:
            ent1 = modifier +" "+ prefix + " "+ tok.text
            prefix = ""
            modifier = ""
            prv_tok_dep = ""
            prv_tok_text = ""      
        
        ## chunk 4
        if tok.dep_.find("obj") == True:
            ent2 = modifier +" "+ prefix +" "+ tok.text  
        
        ## chunk 5  
        # update variables
        prv_tok_dep = tok.dep_
        prv_tok_text = tok.text
        
    #############################################################
    return [ent1.strip(), ent2.strip()]


In [ ]:
entity_pairs = []

for i in sentences:
    entity_pairs.append(get_entities(i))

entity_pairs[0:20]

In [ ]:
#extract relation using the ROOT of the sentence
def get_relation(sent):
    
    # Matcher class object 
    matcher = Matcher(nlp.vocab)
    
    #define the pattern 
    pattern = [{'DEP':'ROOT'}, 
               {'DEP':'prep','OP':"?"},
               {'DEP':'agent','OP':"?"},  
               {'POS':'ADJ','OP':"?"}] 
    
    matcher.add("matching_1", None, pattern) 
    
    matches = matcher(sent)
    k = len(matches) - 1
    
    span = sent[matches[k][1]:matches[k][2]]
    
    return(span.text)

In [ ]:
relations= [get_relation(i) for i in sentences]

In [ ]:
# Analyze syntax
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)

In [ ]:
nlp = spacy.load('en_core_web_sm')


extract_root(sentence): 
    doc = nlp(sentence)
    matcher = Matcher(nlp.vocab)
    root_pattern = [{'DEP':'ROOT'}, 
                    {'DEP':'prep','OP':"?"}, 
                    {'DEP':'agent','OP':"?"}, 
                    {'POS':'ADJ','OP':"?"}] 
    matcher.add("root_matcher", None, root_pattern) 
    matches = matcher(doc) 
    k = len(matches) - 1 
    span = doc[matches[k][1]:matches[k][2]] 
    
    return(span.text)root_words = [] 
for item in sentences_dataframe["sentence"]:
    root_words.append(extract_root(item))